In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from analysis import *
from getters import * 

{'w': Parameter containing:
tensor([[-0.1910, -0.1702, -0.1453,  0.0324,  0.1322,  0.0163, -0.0556, -0.0668,
          0.0829,  0.1951,  0.1547, -0.1675,  0.1080,  0.0485,  0.1479,  0.1093,
         -0.0553, -0.0663, -0.0140,  0.0825, -0.1408,  0.1124,  0.0499, -0.1040,
         -0.1081, -0.0461, -0.0448,  0.0271, -0.0626,  0.0957, -0.1187, -0.0656,
         -0.0442,  0.0033,  0.1118]], requires_grad=True)}
{'w': Parameter containing:
tensor([[-0.1910, -0.1702, -0.1453,  0.0324,  0.1322,  0.0163, -0.0556, -0.0668,
          0.0829,  0.1951,  0.1547, -0.1675,  0.1080,  0.0485,  0.1479,  0.1093,
         -0.0553, -0.0663, -0.0140,  0.0825, -0.1408,  0.1124,  0.0499, -0.1040,
         -0.1081, -0.0461, -0.0448,  0.0271, -0.0626,  0.0957, -0.1187, -0.0656,
         -0.0442,  0.0033,  0.1118]], requires_grad=True)}
[9, 0, 1, 11, 10]
[9, 0, 1, 11, 10]
1.0
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30]
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30]
1.0
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30, 21, 34, 15, 24, 12]
[9, 0,

In [134]:
views=[0, 2, 4, 5]
models=["gcn", "gcn_student", "gcn_student_teacher", "gcn_student_teacher_weight"]
CV=["3Fold", "5Fold", "10Fold"]
run=1
dataset = "gender_data"

def view_reproducibility_analysis(models, CV, views, run):

    view_data_mean = []
    view_data_std = []

    for view in views:
        
        barWidth = 1/(len(models)+1)
        model_result_mean = []
        model_result_std = []
        
        for model in models:
            rep_score, std = view_specific_rep(dataset=dataset, view=view, model=model, run=run, CV=CV)
            model_result_mean.append(rep_score)
            model_result_std.append(std)
        
        view_data_mean.append(model_result_mean)
        view_data_std.append(model_result_std)

    view_data_std.append(list(np.std(view_data_mean, axis=0)))
    view_data_std = np.array(view_data_std).T

    view_data_mean.append(list(np.mean(view_data_mean, axis=0)))

    view_data_mean = np.array(view_data_mean).T
    
    return view_data_mean, view_data_std

In [140]:
mean_all_runs = []
for run in [i for i in range(10)]:
    view_data_mean, view_data_std = view_reproducibility_analysis(models, CV, views, run)
    mean_all_runs.append(view_data_mean)

mean_all_std = np.std(mean_all_runs, axis=0)
mean_all_runs = np.mean(mean_all_runs, axis=0)

In [141]:
mean_all_std

array([[0.06626456, 0.06938275, 0.07135754, 0.06020813, 0.05400997],
       [0.06403305, 0.09971371, 0.06520815, 0.07637891, 0.04003043],
       [0.07261233, 0.09404573, 0.07041557, 0.06954728, 0.03866255],
       [0.08329397, 0.05984097, 0.0743751 , 0.0321431 , 0.03021681]])

In [138]:
pd.DataFrame(mean_all_runs, columns=["View {}".format(i) for i in views]+ ["Average"], index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"])

,View 0,View 2,View 4,View 5,Average
Teacher,0.535972,0.508611,0.536944,0.503194,0.521181
Student,0.481667,0.544167,0.482083,0.538472,0.511597
Student + Teacher,0.484583,0.558889,0.497639,0.531528,0.518160
Student + Teacher + Weight Loss,0.493056,0.529444,0.486528,0.477778,0.496701


# Analysis for each view we create the metric tables across the different models 

In [152]:
def metric_and_view_analysis(models, CV, analysis_type, view, run, dataset_split, dataset, metric):
    """
    Mean of metric for a specific CV -> 3, 5 or 10
    """

    all_data_mean = []
    all_data_std = []
    
    for model in models:
        
        model_results_mean = []
        model_results_std = []
       
        for training_type in CV:
            metrics = extract_metrics(dataset=dataset, model=model, analysis_type=analysis_type, training_type=training_type, view=view, run=run, dataset_split=dataset_split, metric=metric)
            mean = np.mean([metric[-1] for metric in metrics])
            std = np.std([metric[-1] for metric in metrics])
            model_results_mean.append(mean)
            model_results_std.append(std)
        
        all_data_mean.append(model_results_mean)
        all_data_std.append(model_results_std)
    
    return all_data_mean, all_data_std

In [195]:
def view_metric_analysis(models, CV, view, run, metric, dataset):

    view_data_mean = []
    view_data_std = []

    mean, std = metric_and_view_analysis(models=models, 
                                    CV=CV, 
                                    analysis_type="model_assessment", 
                                    view=view, 
                                    run=run, 
                                    dataset= dataset,
                                    dataset_split="val", 
                                    metric=metric)
    view_data_mean.append(mean)
    view_data_std.append(std)
    
    return view_data_mean, view_data_std 

In [207]:
mean_all_runs = []
for run in [i for i in range(10)]:
    view_data_mean, view_data_std = view_metric_analysis(models=models, CV=CV, view=0, run=run, metric="acc", dataset=dataset)
    mean_all_runs.append(view_data_mean)

mean_all_std = np.std(mean_all_runs, axis=0).squeeze()
mean_all_runs = np.mean(mean_all_runs, axis=0).squeeze()

#GET MEAN AND STD ACROSS MEAN OF RUNS
mean_all_runs = np.c_[ mean_all_runs, np.mean(mean_all_runs, axis=1)]     
mean_all_std = np.c_[ mean_all_std, np.std(mean_all_runs, axis=1)]  

mean_all_runs, mean_all_std

(array([[0.60686954, 0.60995643, 0.60961415, 0.60881337],
        [0.57161681, 0.59281791, 0.59920572, 0.58788015],
        [0.57749533, 0.59901104, 0.60518728, 0.59389788],
        [0.54751081, 0.55423042, 0.5613834 , 0.55437487]]),
 array([[0.01529327, 0.01794911, 0.01028801, 0.00119649],
        [0.0249576 , 0.0170908 , 0.01732294, 0.01021208],
        [0.02112357, 0.01607898, 0.0171574 , 0.01027909],
        [0.01541351, 0.01567641, 0.01292345, 0.0049055 ]]))

In [206]:
views=[0, 2, 4, 5]
models=["gcn", "gcn_student", "gcn_student_teacher", "gcn_student_teacher_weight"]
CV=["3Fold", "5Fold", "10Fold"]
run=1
dataset = "gender_data"
dataset_split = ["val", "train", "test"]
metrics = ["acc", "f1", "recall", "precision"]

for view in views:
    for metric in metrics:
        print("____________________________________________________")
        print(f"View: {view} Metric: {metric}")
        mean, std = metric_and_view_analysis(models=models, 
                                            CV=CV, 
                                            analysis_type="model_assessment", 
                                            view=view, 
                                            run=run, 
                                            dataset= dataset,
                                            dataset_split="val", 
                                            metric=metric)
        mean = pd.DataFrame(mean, columns=CV, index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"]).round(3)
        std = pd.DataFrame(std, columns=CV, index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"]).round(3)
        print(mean.astype(str) + u"\u00B1" + std.astype(str))

____________________________________________________
View: 0 Metric: acc
                                       3Fold        5Fold       10Fold
Teacher                          0.602±0.017  0.593±0.038  0.599±0.057
Student                          0.571±0.049  0.603±0.042  0.575±0.042
Student + Teacher                0.582±0.042  0.602±0.042  0.592±0.042
Student + Teacher + Weight Loss   0.532±0.01    0.54±0.04   0.56±0.058
____________________________________________________
View: 0 Metric: f1
                                       3Fold        5Fold       10Fold
Teacher                          0.437±0.046   0.487±0.04  0.519±0.068
Student                          0.381±0.087   0.467±0.08  0.423±0.095
Student + Teacher                  0.35±0.14  0.492±0.071   0.436±0.13
Student + Teacher + Weight Loss   0.17±0.094  0.339±0.077  0.311±0.167
____________________________________________________
View: 0 Metric: recall
                                       3Fold        5Fold       10Fol

In [213]:
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

cv_path_1 = SAVE_DIR_MODEL_DATA+'model_assessment/gcn/weights/W_MainModel_3Fold_gender_data_gcn_run_0_CV_0_view_0_TEST.pickle'
cv_path_2 = SAVE_DIR_MODEL_DATA+'model_assessment/gcn/weights/W_MainModel_3Fold_gender_data_gcn_run_0_CV_0_view_0_TEST_2.pickle'
cv_path_3 = SAVE_DIR_MODEL_DATA+'model_assessment/gcn/weights/W_MainModel_3Fold_gender_data_gcn_run_0_CV_0_view_0_TEST_3.pickle'

with open(cv_path_1,'rb') as f:
    weights_1 = CPU_Unpickler(f).load()

with open(cv_path_2,'rb') as f:
    weights_2 = CPU_Unpickler(f).load()

with open(cv_path_3,'rb') as f:
    weights_2 = CPU_Unpickler(f).load()

print(weights_1)
print(weights_2)

Ks = [5, 10, 15, 20]
teacher_weights = weights_1["w"].squeeze().detach().numpy()
student_weight = weights_2["w"].squeeze().detach().numpy()
for k in Ks:
    top_bio_i = top_biomarkers(teacher_weights, k)
    top_bio_j = top_biomarkers(student_weight, k)
    print(top_bio_i)
    print(top_bio_j)
    print(sim(top_bio_i, top_bio_j))


{'w': Parameter containing:
tensor([[-0.1489,  0.0162, -0.1194, -0.0202, -0.1403, -0.1091, -0.0803,  0.1808,
         -0.0033,  0.0470,  0.0215,  0.0081, -0.0315, -0.1461, -0.0013,  0.1489,
         -0.0930,  0.1272, -0.0292,  0.0739, -0.0244, -0.0515, -0.1271, -0.0550,
          0.0735,  0.0621,  0.0910,  0.1108,  0.0337,  0.1760, -0.0632,  0.0315,
          0.0906,  0.1465, -0.0711]], requires_grad=True)}
{'w': Parameter containing:
tensor([[-0.0586, -0.0967, -0.0213, -0.0423,  0.0074,  0.1082,  0.1008, -0.0700,
          0.1325,  0.1787,  0.0710, -0.1253,  0.1156, -0.0651,  0.0839, -0.0171,
          0.1840,  0.0957, -0.0952, -0.1685,  0.0776, -0.1542,  0.2262,  0.2231,
          0.0125,  0.1908, -0.1573,  0.1821,  0.1211, -0.1819, -0.0794, -0.0089,
          0.0631, -0.0487,  0.0266]], requires_grad=True)}
[7, 29, 15, 0, 33]
[22, 23, 25, 16, 27]
0.0
[7, 29, 15, 0, 33, 13, 4, 17, 22, 2]
[22, 23, 25, 16, 27, 29, 9, 19, 26, 21]
0.2
[7, 29, 15, 0, 33, 13, 4, 17, 22, 2, 27, 5, 16, 26, 3